# Importing helper libraries

In [1]:
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit
import holidays, itertools

%matplotlib inline

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Matplotlib created a temporary cache directory at /tmp/matplotlib-lkh84f2u because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Reading the data

In [2]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS_TRANSFORMED")
data

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7fe8d614c970>
Please close the connection after use!
Reading dataframe from snowflake native connector


,ARRIVAL_DATE_TRANSFORMED,RESERVATION_STATUS_DATE_TRANSFORMED,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR,TRANSCATION_IDS,CUST_ID,ROOM_LIMIT
0,2023-08-03,2023-08-06,City Hotel,0,86,2023,8,31,3,03-08-2023,Check.Out,06-08-2023,3,3.0,0,3,3,0,0,3,1,3,BB,DEU,Online TA,TA/TO,0,0,D,D,No Deposit,0,Transient,195.00,B20230803343215,C0106952,135
1,2023-08-02,2023-08-06,Resort Hotel,0,10,2023,8,31,2,02-08-2023,Check.Out,06-08-2023,4,4.0,0,4,2,0,0,2,1,4,BB,PRT,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,228.50,B20230802911470,C0106953,360
2,2023-08-03,2023-08-06,City Hotel,0,180,2023,8,31,3,03-08-2023,Check.Out,06-08-2023,3,3.0,0,3,2,0,0,2,1,3,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,80.10,B20230803481168,C0106954,460
3,2023-07-29,2023-08-06,City Hotel,0,148,2023,7,30,29,29-07-2023,Check.Out,06-08-2023,8,8.0,2,6,2,0,0,2,1,8,SC,FRA,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,121.88,B20230729335744,C0106955,460
4,2023-08-05,2023-08-06,City Hotel,0,0,2023,8,31,5,05-08-2023,Check.Out,06-08-2023,1,1.0,0,1,1,0,0,1,1,1,SC,FRA,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,179.00,B20230805458332,C0106956,460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109706,2023-08-04,2023-08-06,City Hotel,0,11,2023,8,31,4,04-08-2023,Check.Out,06-08-2023,2,2.0,0,2,2,0,0,2,1,2,SC,SLV,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,160.00,B20230804347306,C0106947,460
109707,2023-07-30,2023-08-06,Resort Hotel,0,300,2023,7,31,30,30-07-2023,Check.Out,06-08-2023,7,7.0,2,5,2,0,0,2,1,7,BB,PRT,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,149.81,B20230730168488,C0106948,360
109708,2023-07-29,2023-08-06,City Hotel,0,69,2023,7,30,29,29-07-2023,Check.Out,06-08-2023,8,8.0,2,6,2,0,0,2,1,8,SC,RUS,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient.Party,126.00,B20230729444159,C0106949,460
109709,2023-08-01,2023-08-06,City Hotel,0,147,2023,8,31,1,01-08-2023,Check.Out,06-08-2023,5,5.0,0,5,3,0,0,3,1,5,BB,SWE,Online TA,TA/TO,0,0,D,D,No Deposit,0,Transient,175.50,B20230801923755,C0106950,135


In [4]:
data.columns = map(lambda x: str(x).lower(), data.columns)


In [5]:
hotels = data['hotel'].unique()


In [6]:
from scipy.optimize import curve_fit

# Define the demand curve function
def demand_curve(x, a, b, c, d, max_demand):
    demand = a * np.exp(-b * x) + c
    demand = np.where(x <= max_demand, np.minimum(demand, max_demand), demand)
    return demand + d

In [7]:
def revenue(price):
    return price * demand_curve(price, a_fit, b_fit,c_fit,d_fit,max_demand)

In [8]:
from scipy.optimize import brentq

def demand_to_price(num_rooms, a, b, c, d, max_demand):
    def root_func(x):
        return num_rooms - (a * np.exp(-b * x) + c)

    try:
        price = brentq(root_func, 0, 200)  # Adjust the interval bounds as needed
    except ValueError:
        # Fallback to default price if no root is found
        price_range=(0, 200)
        price = np.random.uniform(*price_range)

    return price

In [9]:
def generate_holiday_dates(start_year, end_year):
    holidays = {}
    for year in range(start_year, end_year + 1):
        holidays[datetime.date(year, 1, 1)] = 'new_year'
        easter_date = easter(year)
        holidays[easter_date] = 'easter'
        holidays[datetime.date(year, 12, 25)] = 'christmas'
    return holidays

holidays = generate_holiday_dates(2020, 2023)

In [11]:
def data_conversion(data):
    import holidays
    hotels = data['hotel'].unique()
    room_types = data['reserved_room_type'].unique()
    expanded_df = pd.DataFrame()
    for _, row in data.iterrows():
        num_stay_dates = row['total_rns']
        try:
            # Create a row for each stay date
            expanded_booking = pd.DataFrame({
                'hotel': row['hotel'],
                'room_type': row['reserved_room_type'], 
                'arrival_date': pd.date_range(start=row['expected_arrival_date'], periods=num_stay_dates),
                'total_rns': 1,
                'adr': row['adr'],
                'room_limit': row['room_limit']
            })

            # Append the stay date information to the new dataframe
            expanded_df = pd.concat([expanded_df, expanded_booking], ignore_index=True)
        except ValueError as e:
            print(f"Error processing booking for {row['hotel']} on {row['expected_arrival_date']} : {num_stay_dates} {e}")
    expanded_df = expanded_df.sort_values('arrival_date')
    expanded_df = expanded_df.reset_index(drop=True)
    expanded_df['adr']= np.round(expanded_df['adr'], 2)
           
    expanded_df['dow'] = expanded_df.arrival_date.dt.strftime('%A')
    expanded_df['month'] = expanded_df.arrival_date.dt.strftime('%B')

    return expanded_df

data['total_rns'] = data['stays_in_week_nights'] + data['stays_in_weekend_nights']


In [16]:
data = data[(data['is_canceled'] == 0) & (data['reservation_status'] !='No-Show')] 
data = data[(data.market_segment != 'Complementary') ]
data = data[(data.reserved_room_type == 'A') |(data.reserved_room_type == 'D') | (data.reserved_room_type == 'E')]

converted_df = data_conversion(data[['hotel', 'reserved_room_type', 'expected_arrival_date', 'adr', 'room_limit', 'total_rns']])
converted_df

Error processing booking for City Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for Resort Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for City Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for City Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for City Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for Resort Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for City Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for City Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for Resort Hotel on 29-02-2022 : 1 day is out of range for month
Error processing booking for City Hotel on 29-02-2022 : 2 day is out of range for month
Error processing booking for Resort Hotel on 29-02-2022 : 2 day is out of range for month
Error processing booking

,hotel,room_type,arrival_date,total_rns,adr,room_limit,dow,month
0,City Hotel,A,2021-01-07,1,76.50,460,Thursday,January
1,City Hotel,A,2021-01-07,1,101.50,460,Thursday,January
2,City Hotel,A,2021-01-07,1,101.50,460,Thursday,January
3,Resort Hotel,A,2021-01-07,1,98.00,360,Thursday,January
4,City Hotel,A,2021-01-07,1,80.00,460,Thursday,January
...,...,...,...,...,...,...,...,...
232598,Resort Hotel,A,2023-12-18,1,169.00,360,Monday,December
232599,Resort Hotel,A,2023-12-18,1,40.95,360,Monday,December
232600,Resort Hotel,A,2023-12-18,1,151.00,360,Monday,December
232601,Resort Hotel,E,2023-12-19,1,158.00,100,Tuesday,December


In [20]:
def format_data(expanded_df):
    results = pd.DataFrame(columns=['month', 'hotel','room_limit', 'room_type', 'dow', 'optimal_rate', 'expected_rn','expected_rev','optimal_rate_lim_inv'])
    hotel_types = ['Resort Hotel', 'City Hotel']
    room_types = ['A', 'D', 'E']
    daily_rns= expanded_df.groupby(['arrival_date','dow','month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns':'sum'}).reset_index() # ge total stays per day

    daily_rns = daily_rns.groupby(['dow','month', 'hotel', 'room_type']).agg({'room_limit': 'mean','total_rns':['sum','mean','median']}).reset_index() # get Rns metrics by Dow & Month

    daily_rns.columns = ['_'.join(col) for col in daily_rns.columns] #remove multi level column
    adr_frequency = expanded_df.groupby(['dow','month','adr', 'hotel', 'room_type']).agg({'room_limit': 'mean','total_rns':'sum'})
    adr_frequency.reset_index(inplace=True)
    merged_df = pd.merge(adr_frequency, daily_rns,how='left',left_on=['dow','month', 'hotel', 'room_type'], right_on=['dow_','month_', 'hotel_', 'room_type_'],suffixes=('_act', '_tot'))

    merged_df = merged_df.drop(['dow_','month_'],axis=1)

    merged_df['probability'] = merged_df['total_rns']/merged_df['total_rns_sum']
    merged_df['expected_rns'] = merged_df['probability'] * merged_df['total_rns_median']
    merged_df = merged_df.sort_values(by=['dow', 'month', 'adr'], ascending=[True, True, False])
    merged_df['expected_demand']=merged_df.groupby(['dow', 'month'])['expected_rns'].cumsum()
    merged_df['expected_rev'] = merged_df['adr']* merged_df['expected_demand']
    results = pd.DataFrame(columns=['month', 'hotel','room_limit', 'room_type', 'dow', 'optimal_rate', 'expected_rn','expected_rev','optimal_rate_lim_inv'])
    months = merged_df.month.unique()
    dow = merged_df.dow.unique()
    for hotel in hotel_types:
        for room_type in room_types:
            for month in months:
                for day in dow:
                    # Get data for the specific combination
                    data_subset = merged_df[(merged_df['dow'] == day) & 
                                            (merged_df['hotel'] == hotel) & 
                                            (merged_df['room_type'] == room_type) & 
                                            (merged_df['month'] == month)].reset_index()

                    if data_subset.empty:
                        continue

                    # Remove outliers
                    mean = data_subset['adr'].mean()
                    std_dev = data_subset['adr'].std()
                    data_subset['z_scores'] = np.abs((data_subset['adr'] - mean) / std_dev)
                    data_subset = data_subset[data_subset['z_scores'] <= 2]

                    # Fit demand curve
                    x_data = data_subset['adr'].values
                    y_data = data_subset['expected_demand'].values
                    try:
                        initial_guess = [1, 0.01, 1, 1, 25.0]
                        bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])
                        maxfev = 10000  # Increase the number of maximum function evaluations
                        params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess, maxfev=maxfev)
                    except RuntimeError as e:
                        print(f"Error fitting demand curve for {hotel}, {room_type}, {month}, {day}: {e}")
                        continue
                    except ValueError as e:
                        print(f"Error fitting demand curve for {hotel}, {room_type}, {month}, {day}: {e}")
                        continue
                    a_fit, b_fit, c_fit, d_fit, max_demand = params

                    # Optimize revenue
                    def revenue(price):
                        return price * demand_curve(price, a_fit, b_fit, c_fit, d_fit, max_demand)

                    objective = lambda price: -revenue(price)
                    optimize = minimize_scalar(objective, bounds=(45, 200), method='bounded')
                    optimal_price = optimize.x
                    max_revenue = -optimize.fun
                    expected_rns = demand_curve(optimal_price, a_fit, b_fit, c_fit, d_fit, max_demand)

                    optimal_rate_lim_inv = demand_to_price(data_subset['room_limit'].mean(), a_fit, b_fit, c_fit, d_fit, max_demand)

                    new_row = pd.DataFrame({'hotel': hotel,
                                            'room_type': room_type,
                                            'room_limit': data_subset['room_limit'].mean(),
                                            'month': month,
                                            'dow': day,
                                            'optimal_rate': optimal_price,
                                            'expected_rev': max_revenue,
                                            'expected_rn': expected_rns,
                                            'optimal_rate_lim_inv': optimal_rate_lim_inv}, index=[0])
                    results = pd.concat([results, new_row], ignore_index=True)
    results['optimal_rate'] = results['optimal_rate'].round()
    results['optimal_rate_lim_inv'] = results['optimal_rate_lim_inv'].round()

    results['expected_rn'] = results['expected_rn'].round().astype(int)
    results['expected_rev'] = results['expected_rev'].round()
    combinations = list(itertools.product(hotel_types, room_types))
    combinations_df = pd.DataFrame(combinations, columns=['hotel', 'room_type'])
    month_dict = {month: index for index, month in enumerate(pd.date_range('2020-01-01', periods=12, freq='M').strftime('%B'), 1)}


    new_data = pd.DataFrame()

    for year in range(2020, 2024):
        for month in month_dict.values():
            start_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1)
            end_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1) + pd.offsets.MonthEnd(0)
            date_range = pd.date_range(start_date, end_date, freq='D')
            df = pd.DataFrame(date_range, columns=['arrival_date'])
            df['dow'] = df['arrival_date'].dt.day_name()
            df['month'] = df['arrival_date'].dt.month_name()

            result_df = df.assign(key=1).merge(combinations_df.assign(key=1), on='key').drop('key', axis=1)
            new_data = pd.concat([new_data, result_df], ignore_index=True)

    final_data = pd.merge(new_data, results, how='left', on=['dow', 'hotel', 'room_type', 'month'])
    final_data['room_limit'] = final_data['room_limit'].astype(int)
    return final_data


In [ ]:
format_data(converted_df)

Error fitting demand curve for Resort Hotel, A, August, Monday: Optimal parameters not found: The maximum number of function evaluations is exceeded.
Error fitting demand curve for Resort Hotel, A, August, Thursday: Optimal parameters not found: The maximum number of function evaluations is exceeded.
